In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.corpus import stopwords
#pd.options.display.max_rows = 999
from textblob import TextBlob
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as np

In [3]:
df_test = pd.read_csv('test.csv')
df = pd.read_csv('train.csv')

In [4]:
df.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [5]:
df['length']=df['tweet'].apply(lambda x: len(x.split(' ')))
#df.head()
df_test['length']=df_test['tweet'].apply(lambda x: len(x.split(' ')))


In [6]:
df['lenChar']=df['tweet'].apply(lambda x: len(x))
#df.head()

In [7]:
df['avg_len']=df['lenChar']/df['length']
#df.head()

In [8]:
stop_words=stopwords.words('english')
df['#_stopwords']=df['tweet'].apply(lambda x: len([y for y in x.split(' ') if y in stop_words]))
#df.head()
df_test['#_stopwords']=df_test['tweet'].apply(lambda x: len([y for y in x.split(' ') if y in stop_words]))


In [9]:
df['#_numerics']=df['tweet'].apply(lambda x: len([y for y in x.split(' ') if y.isdigit()]))
#df.head()

In [10]:
#%%time
df['tweet']=df['tweet'].apply(lambda x: x.lower())
#df.head()
df_test['tweet']=df_test['tweet'].apply(lambda x: x.lower())


In [11]:
df['tweet']=df['tweet'].str.replace('[^\w\s]','')
#df.head()
df_test['tweet']=df_test['tweet'].str.replace('[^\w\s]','')


In [12]:
df['tweet']=df['tweet'].apply(lambda x: ' '.join(y for y in x.split(' ') if y not in stop_words))
#df.head()
df_test['tweet']=df_test['tweet'].apply(lambda x: ' '.join(y for y in x.split(' ') if y not in stop_words))


In [13]:
freq=pd.Series(' '.join(df['tweet']).split(' ')).value_counts()
freq_test=pd.Series(' '.join(df_test['tweet']).split(' ')).value_counts()

In [14]:
freqent_words=freq[1:3]
freqent_words=list(freqent_words.index)
df['tweet']=df['tweet'].apply(lambda x: ' '.join(y for y in x.split(' ') if y not in freqent_words))
#df.head()
freqent_words_test=freq_test[1:3]
freqent_words_test=list(freqent_words_test.index)
df_test['tweet']=df_test['tweet'].apply(lambda x: ' '.join(y for y in x.split(' ') if y not in freqent_words_test))

In [15]:
rare_words=freq[freq==1]
rare_words=list(rare_words.index)
df['tweet']=df['tweet'].apply(lambda x: ' '.join(y for y in x.split(' ') if y not in rare_words))
#df.head()
rare_words_test=freq_test[freq_test==1]
rare_words_test=list(rare_words_test.index)
df_test['tweet']=df_test['tweet'].apply(lambda x: ' '.join(y for y in x.split(' ') if y not in rare_words_test))

In [16]:
'''from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
df['tweet'] = df['tweet'].apply(lambda x: " ".join([lemmatizer.lemmatize(x) for word in x.split()]))
df['tweet'].head()'''

'from nltk.stem import WordNetLemmatizer\nlemmatizer = WordNetLemmatizer()\ndf[\'tweet\'] = df[\'tweet\'].apply(lambda x: " ".join([lemmatizer.lemmatize(x) for word in x.split()]))\ndf[\'tweet\'].head()'

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word',stop_words= 'english',ngram_range=(1,1))
vect = tfidf.fit_transform(df['tweet'])
v=vect.toarray()

tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word',stop_words= 'english',ngram_range=(1,1))
vect_test = tfidf.fit_transform(df_test['tweet'])
v_test=vect_test.toarray()

In [18]:
#print(vect)

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(lowercase=True,analyzer = "word")
c_bow = bow.fit_transform(df['tweet'])
c=c_bow.toarray()

bow = CountVectorizer(lowercase=True,analyzer = "word")
c_bow_test = bow.fit_transform(df_test['tweet'])
c_test=c_bow_test.toarray()

In [20]:
#print(c_bow)

In [21]:
from sklearn.cross_validation import train_test_split

'''X_train, X_test, y_train, y_test  = train_test_split(
        v, 
        df['label'],
        train_size=0.80, 
        random_state=1234)'''

X_train=v
y_train=df['label']
X_test=v_test

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [22]:
'''from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression()
log_model = log_model.fit(X=X_train, y=y_train)
y_pred = log_model.predict(X_test)
#accuracy_score(y_test, predict)'''

'from sklearn.linear_model import LogisticRegression\nlog_model = LogisticRegression()\nlog_model = log_model.fit(X=X_train, y=y_train)\ny_pred = log_model.predict(X_test)\n#accuracy_score(y_test, predict)'

In [23]:
from sklearn.ensemble import RandomForestClassifier

random_forest=RandomForestClassifier(n_estimators=58)
random_forest.fit(X_train, y_train)
predict=random_forest.predict(X_test)
#accuracy_score(y_test, predict)

In [24]:
pred=pd.Series(predict)
pred.to_csv('prediction.csv')